In [30]:
import pandas as pd
import requests
import copy
from bs4 import BeautifulSoup

In [36]:
# Read in the html file
url = 'https://www.mlbtraderumors.com/2022/10/mlb-projected-arbitration-salaries-2023.html'


response = requests.get(url)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
      
    # Find the div with class 'entry-content' and get all <li> elements
    entry_content_div = soup.find('div', class_='entry-content')
    list_items = entry_content_div.find_all('li')

    # Extract text content from each <li> element
    items_text = [li.get_text(strip=True) for li in list_items]

    # Create a pandas DataFrame
    df = pd.DataFrame({'Items': items_text})

    # Print or do further processing with the DataFrame
    print(df)

else:
    print(f"Unable to find on the page.")        

                              Items
0    Griffin Canning(3.075): $1.1MM
1       Luis Rengifo(3.043): $2.4MM
2       Jaime Barria(3.035): $1.2MM
3        Chad Wallach(3.030): $800K
4        Jared Walsh(3.010): $2.7MM
..                              ...
299  Kyle Higashioka(4.005): $1.7MM
300     Tim Locastro(3.122): $1.2MM
301    Nestor Cortes(3.094): $3.5MM
302       Jose Trevino(3.063): $2MM
303     Michael King(3.004): $1.2MM

[304 rows x 1 columns]


In [41]:
data = copy.deepcopy(df)
data['Player'] = data['Items'].str.split('(').str[0]
data['Service'] = data['Items'].str.split('(').str[1].str.split(')').str[0]
data['SalRaw'] = data['Items'].str.split('$').str[-1]
# Function to convert salary values to integers
def convert_salary_to_int(salary_str):
    try:
        if 'K' in salary_str:
            return int(float(salary_str.replace('K', '')) * 1000)
        elif 'MM' in salary_str:
            return int(float(salary_str.replace('MM', '')) * 1000000)
        else:
            return int(float(salary_str))
    except ValueError:
        return None  # Return None for rows with non-convertible values

# Apply the conversion function to the 'Salary' column
data['TR Pred Salary'] = data['SalRaw'].apply(convert_salary_to_int)
# split_names = data['Player'].str.split(' ', n=1, expand=True)
# data[['first_name', 'last_name']]  = split_names
data.drop(columns=['Items', 'SalRaw'], inplace=True)
data

,Player,Service,TR Pred Salary
0,Griffin Canning,3.075,1100000.0
1,Luis Rengifo,3.043,2400000.0
2,Jaime Barria,3.035,1200000.0
3,Chad Wallach,3.030,800000.0
4,Jared Walsh,3.010,2700000.0
...,...,...,...
299,Kyle Higashioka,4.005,1700000.0
300,Tim Locastro,3.122,1200000.0
301,Nestor Cortes,3.094,3500000.0
302,Jose Trevino,3.063,2000000.0


In [42]:
data.to_csv('data/mlbtr_2023_preds.csv', index=False)